<a href="https://colab.research.google.com/github/Ghustavo516/Trainer-IA/blob/main/Fine_Tunning_QA_Model_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#Baixe as dependencias
!pip install transformers datasets torch

In [15]:
#Importe as dependencias
import pandas as pd
from transformers import BertTokenizerFast, BertForQuestionAnswering, Trainer, TrainingArguments
from datasets import Dataset
import torch
import os

In [16]:
os.environ["WANDB_DISABLED"] = "true"

In [18]:
# Carregar o arquivo CSV com pandas
df = pd.read_csv('./teste.csv')

# Preprocessamento do CSV para o formato correto
def preprocess_data(examples):
    questions = examples['question']
    contexts = examples['context']
    answers = examples['answer']

    # Tokenizar perguntas e contexto
    tokenized_inputs = tokenizer(questions, contexts, truncation=True, padding='max_length', max_length=512)

    # Inicializar listas para as posições de início e fim
    start_positions = []
    end_positions = []

    for i, answer in enumerate(answers):
        context = contexts[i]

        # Verificar se a resposta existe no contexto
        start_idx = context.find(answer)
        if start_idx != -1:
            end_idx = start_idx + len(answer)
            # Ajustar as posições para os tokens
            token_start = tokenized_inputs.char_to_token(i, start_idx)
            token_end = tokenized_inputs.char_to_token(i, end_idx - 1)

            # Caso os tokens não sejam encontrados, atribua -1
            if token_start is None or token_end is None:
                start_positions.append(-1)
                end_positions.append(-1)
            else:
                start_positions.append(token_start)
                end_positions.append(token_end)
        else:
            # Se a resposta não for encontrada no contexto, atribua -1
            start_positions.append(-1)
            end_positions.append(-1)

    # Adicionar as posições ao tokenizado
    tokenized_inputs['start_positions'] = start_positions
    tokenized_inputs['end_positions'] = end_positions
    return tokenized_inputs

In [ ]:
# Tokenizer e modelo - <Substitua o modelo pelo o que voce necessitar>
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

In [20]:
# Converter o DataFrame para Dataset Hugging Face
dataset = Dataset.from_pandas(df)

In [ ]:
# Processar os dados
dataset = dataset.map(preprocess_data, batched=True)

In [22]:
# Dividir em treino e validação
split_dataset = dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset["train"]
validation_dataset = split_dataset["test"]

In [ ]:
# Configurar argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./qa_model", #Diretorio onde será salvo o seu modelo
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,  # Ajuste conforme necessário <4, 8, 12, 16>
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    push_to_hub=False,
    report_to="none",
    logging_dir="./logs", # Armazena os logs para ser acompanhados durante o treinamento
)



In [ ]:
# 5. Criar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer
)

In [ ]:
# Iniciar o treinamento
trainer.train()

# Salvando o modelo no diretório especificado
trainer.save_model()

# Caso queira salvar o tokenizador junto com o modelo:
model.save_pretrained(training_args.output_dir, safe_serialization=True)
tokenizer.save_pretrained(training_args.output_dir)